<a href="https://colab.research.google.com/github/hamideshahrabi/editorial_assistant/blob/main/run_in_colabFromGithub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Cell 1: Mount Google Drive and set up project
from google.colab import drive
import os
from pathlib import Path
import json

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Create project directory in Google Drive
project_dir = '/content/drive/MyDrive/editorial_assistant'
os.makedirs(f'{project_dir}/data', exist_ok=True)
os.makedirs(f'{project_dir}/src/api', exist_ok=True)

# Create requirements.txt
requirements = """fastapi>=0.104.1
uvicorn>=0.24.0
sentence-transformers>=2.2.2
faiss-cpu>=1.7.4
transformers>=4.35.2
torch>=2.1.1
numpy>=1.26.0
pydantic>=2.7.4
requests>=2.31.0
python-multipart>=0.0.6"""

with open(f'{project_dir}/requirements.txt', 'w') as f:
    f.write(requirements)

# Create articles.json
articles = [
    {
        "content_id": "article1",
        "content_headline": "CBC N.L. launches campaign to support food banks",
        "content_type": "news",
        "content_publish_time": "2024-01-15T10:00:00Z",
        "content_last_update": "2024-01-15T10:00:00Z",
        "content_categories": ["Community", "Food Security"],
        "content_tags": ["food bank", "community support", "CBC N.L."],
        "content_word_count": 450,
        "content_department_path": "/news/local",
        "body": "CBC N.L. has launched a campaign to support food banks across the province. The initiative, in partnership with the Community Food Sharing Association, aims to raise awareness and collect donations for food banks that are seeing increased demand. With rising food costs and economic challenges, many families are turning to food banks for support. The campaign will run for the next month, with special programming and community events planned throughout the province."
    },
    {
        "content_id": "article2",
        "content_headline": "New guidelines for social media reporting",
        "content_type": "policy",
        "content_publish_time": "2024-01-10T14:30:00Z",
        "content_last_update": "2024-01-10T14:30:00Z",
        "content_categories": ["Policy", "Social Media"],
        "content_tags": ["social media", "reporting guidelines", "journalism"],
        "content_word_count": 600,
        "content_department_path": "/news/policy",
        "body": "CBC has released updated guidelines for social media reporting. The new policy emphasizes accuracy, transparency, and responsible use of social media platforms. Journalists are required to verify information from social media sources before reporting, maintain professional boundaries, and clearly distinguish between personal and professional accounts. The guidelines also address the use of user-generated content and the importance of protecting sources who share information through social media."
    }
]

articles_path = f'{project_dir}/data/articles.json'
print("\nWriting articles.json...")
with open(articles_path, 'w') as f:
    json.dump(articles, f, indent=2)

# Verify articles.json content
print("\nVerifying articles.json content...")
with open(articles_path, 'r') as f:
    content = f.read()
    print(f"Content length: {len(content)} characters")
    print(f"First 100 characters: {content[:100]}")
    print(f"Last 100 characters: {content[-100:]}")
    # Verify JSON is valid
    try:
        json.loads(content)
        print("JSON is valid")
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON - {str(e)}")

# Create policies.txt
policies = '''CBC Editorial Guidelines: Anonymous Sources

Anonymous sources should only be used when:
1. The information is of significant public interest
2. The source would face serious consequences if identified
3. The information cannot be obtained through on-the-record sources

When using anonymous sources:
- Verify the information through multiple sources
- Clearly explain to readers why anonymity was granted
- Use descriptive terms (e.g., "senior government official" instead of just "source")
- Document the source's credentials and relationship to the story

CBC Editorial Guidelines: Headlines

Headlines should:
1. Be accurate and reflect the content
2. Avoid sensationalism
3. Use clear, concise language
4. Include relevant keywords for SEO
5. Follow CBC's style guide for capitalization and punctuation

CBC Editorial Guidelines: Social Media

When creating content for social media:
1. Keep summaries concise and engaging
2. Use appropriate hashtags
3. Include key information in the first 280 characters
4. Maintain CBC's voice and tone
5. Ensure accuracy and fairness'''

policies_path = f'{project_dir}/data/policies.txt'
print("\nWriting policies.txt...")
with open(policies_path, 'w') as f:
    f.write(policies)

# Write the corrected main.py file
main_py_content = '''import json
from typing import List, Dict, Any
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import faiss
import numpy as np
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pathlib import Path
import logging
import re

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Initialize components
model = None
vector_store = None
articles = []
policies = []
policy_sections = []

def extract_specific_details(text, question):
    """Extract specific details like numbers and percentages from text."""
    details = []

    # Look for numbers and percentages
    number_pattern = r'\\$?\\d+(?:,\\d+)*(?:\\.\\d+)?%?'
    numbers = re.findall(number_pattern, text)

    # Look for specific phrases
    if "raised" in question.lower():
        for num in numbers:
            if "$" in num:
                details.append(f"Amount raised: {num}")

    if "increase" in question.lower():
        for num in numbers:
            if "%" in num:
                details.append(f"Expected increase: {num}")

    if "families" in question.lower():
        for num in numbers:
            if num.isdigit():
                details.append(f"Number of families: {num}")

    return details

def split_policies(text):
    """Split policies into sections based on headers."""
    sections = []
    current_section = []
    current_title = None

    for line in text.split("\\n"):
        if line.startswith("CBC Editorial Guidelines:"):
            if current_section and current_title:
                sections.append({
                    "title": current_title,
                    "content": "\\n".join(current_section).strip()
                })
            current_title = line.strip()
            current_section = []
        else:
            current_section.append(line)

    if current_section and current_title:
        sections.append({
            "title": current_title,
            "content": "\\n".join(current_section).strip()
        })

    return sections

def initialize_components():
    global model, vector_store, articles, policies, policy_sections

    # Load model
    logger.info("Loading model...")
    model = SentenceTransformer("all-MiniLM-L6-v2")
    if torch.cuda.is_available():
        model = model.to("cuda")
        logger.info(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        logger.info("Using CPU")

    # Load data
    logger.info("Loading data...")
    data_dir = Path("data")

    # Load articles
    logger.info("Loading articles...")
    with open(data_dir / "articles.json", "r") as f:
        articles = json.load(f)
    logger.info(f"Successfully loaded {len(articles)} articles")

    # Load and split policies
    logger.info("Loading policies...")
    with open(data_dir / "policies.txt", "r") as f:
        policies = f.read()
    policy_sections = split_policies(policies)
    logger.info(f"Split policies into {len(policy_sections)} sections")

    # Create vector store
    logger.info("Creating vector store...")
    texts = []
    sources = []

    # Add articles
    for article in articles:
        texts.append(article["body"])
        sources.append({
            "type": "article",
            "title": article["content_headline"]
        })

    # Add policy sections
    for section in policy_sections:
        texts.append(section["content"])
        sources.append({
            "type": "policy",
            "title": section["title"]
        })

    embeddings = model.encode(texts)
    logger.info(f"Created embeddings of shape: {embeddings.shape}")

    # Initialize FAISS index
    dimension = embeddings.shape[1]
    vector_store = faiss.IndexFlatL2(dimension)
    vector_store.add(embeddings.astype("float32"))
    logger.info("Vector store created")

    # Store sources and texts for later use
    app.state.sources = sources
    app.state.texts = texts
    return True

class Question(BaseModel):
    question: str

@app.get("/")
async def root():
    logger.info("Root endpoint called")
    return {"status": "ok", "message": "CBC Editorial Assistant API is running"}

@app.post("/api/qa")
async def answer_policy_question(question: Question):
    global model, vector_store

    if model is None or vector_store is None:
        success = initialize_components()
        if not success:
            raise HTTPException(status_code=500, detail="Failed to initialize components")

    try:
        # Encode question
        question_embedding = model.encode([question.question])

        # Search for similar content
        D, I = vector_store.search(question_embedding.astype("float32"), k=5)  # Increased from 3 to 5

        # Get relevant text
        citations = []
        seen_texts = set()  # To avoid duplicate citations
        seen_sources = set()  # To avoid duplicate sources

        for idx in I[0]:
            source = app.state.sources[idx]
            text = app.state.texts[idx]

            # Skip if we've already seen this text or source
            if text in seen_texts or source["title"] in seen_sources:
                continue
            seen_texts.add(text)
            seen_sources.add(source["title"])

            # For articles, try to find the most relevant paragraph
            if source["type"] == "article":
                paragraphs = text.split("\\n\\n")
                most_relevant_para = max(paragraphs, key=lambda p: model.encode([p])[0] @ question_embedding[0])

                # Extract specific details if relevant
                details = extract_specific_details(most_relevant_para, question.question)
                if details:
                    most_relevant_para = "\\n".join([most_relevant_para] + details)

                # Only add if the paragraph is relevant to the question
                if model.encode([most_relevant_para])[0] @ question_embedding[0] > 0.3:  # Relevance threshold
                    citations.append({
                        "source": f"CBC Article: {source['title']}",
                        "text": most_relevant_para
                    })
            else:
                # For policies, only add if the section is relevant to the question
                if model.encode([text])[0] @ question_embedding[0] > 0.3:  # Relevance threshold
                    citations.append({
                        "source": source["title"],
                        "text": text
                    })

        if not citations:
            raise HTTPException(status_code=404, detail="No relevant information found")

        # Return the most relevant text as the answer
        answer = citations[0]["text"]

        # Return the response
        return {
            "answer": answer,
            "citations": citations
        }
    except Exception as e:
        logger.error(f"Error in QA endpoint: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    import uvicorn
    logger.info("Starting server...")
    initialize_components()
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")
'''

# Write the file
with open('/content/drive/MyDrive/editorial_assistant/src/api/main.py', 'w') as f:
    f.write(main_py_content)

print("main.py has been updated successfully!")



# Create __init__.py files
with open(f'{project_dir}/src/__init__.py', 'w') as f:
    f.write('')

with open(f'{project_dir}/src/api/__init__.py', 'w') as f:
    f.write('')

print("\nCreated project structure in Google Drive:")
print(f"Project directory: {project_dir}")
print("\nVerifying directory structure:")
print(f"Directory contents: {os.listdir(project_dir)}")
print(f"src directory contents: {os.listdir(f'{project_dir}/src')}")
print(f"src/api directory contents: {os.listdir(f'{project_dir}/src/api')}")
print(f"data directory contents: {os.listdir(f'{project_dir}/data')}")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Writing articles.json...

Verifying articles.json content...
Content length: 1982 characters
First 100 characters: [
  {
    "content_id": "article1",
    "content_headline": "CBC N.L. launches campaign to support f
Last 100 characters:  content and the importance of protecting sources who share information through social media."
  }
]
JSON is valid

Writing policies.txt...
main.py has been updated successfully!

Created project structure in Google Drive:
Project directory: /content/drive/MyDrive/editorial_assistant

Verifying directory structure:
Directory contents: ['test_server.py', 'editorial_assistant.html', 'run_server.py', 'src', 'data', 'server.log', 'editorial_assistant', 'requirements.txt']
src directory contents: ['__pycache__', 'api', '__init__.py']
src/api directory contents: ['__pycache__', '__init__.py', 'main.py']
dat

In [9]:
import json

# Create a clean article
article = {
    "content_id": "1.6272172",
    "content_headline": "CBC N.L. launches annual Make the Season Kind campaign to support food banks",
    "content_type": "Text",
    "content_publish_time": "2021-12-06T04:30:00",
    "content_last_update": "2021-12-06T04:30:00",
    "content_edits": "0",
    "content_categories": [
        {"content_category": "Holidays"},
        {"content_category": "Political campaigns"},
        {"content_category": "Local food"},
        {"content_category": "Food banks"}
    ],
    "content_tags": [
        {"type": "generic", "name": "CBC Feed NL"},
        {"type": "generic", "name": "CBC Feed NL Day"},
        {"type": "generic", "name": "Food banks"},
        {"type": "generic", "name": "Food donations"},
        {"type": "generic", "name": "donations"},
        {"type": "location", "name": "Newfoundland and Labrador"},
        {"type": "organization", "name": "CBC"},
        {"type": "organization", "name": "Salvation Army"},
        {"type": "person", "name": "Rene Loveless"}
    ],
    "content_word_count": "288",
    "content_department_path": "News/Canada/Nfld. & Labrador",
    "body": "CBC N.L. is kicking off the annual Make the Season Kind campaign for the month of December with special programming and a virtual coming together in support of food banks across the province. This year, CBC N.L. is continuing to partner with the Community Food Sharing Association, an organization that distributes to food banks across Newfoundland and Labrador year-round to help keep pantries stocked for people in need. Included is the annual Feed N.L. Day on Dec. 10, which in 2020 helped raise $193,815 for local food banks. Some community organizations across Newfoundland and Labrador are experiencing increased calls for help leading up to the holidays. Bridges to Hope and Food First N.L. both say more first-time food bank users are reaching out. Maj. Rene Loveless of the Salvation Army said families and individuals are making tough decisions every day on how to put food on their tables while trying to make ends meet. \"When it comes to the increased cost of supplying food for their families, that's certainly a significant factor,\" he said. \"And it's a significant factor for us a provider as well. Our food dollar doesn't take us as far as it used to. So that's something that we're certainly seeing as well. Loveless said donations to his organization are going well so far this year. Last year, the Salvation Army helped 2,000 families over the holidays, he said, but added it's expected the need is going to increase by 15 per cent this time around. The annual Make the Season Kind campaign will be virtual again this year. The public can support the Community Food Sharing Association by making a donation online."
}

# Write to file
articles_path = '/content/drive/MyDrive/editorial_assistant/data/articles.json'
with open(articles_path, 'w') as f:
    json.dump([article], f, indent=2)

# Verify the file
print("Verifying articles.json content:")
with open(articles_path, 'r') as f:
    content = f.read()
    print("File content length:", len(content))
    print("First 100 characters:", content[:100])
    print("Last 100 characters:", content[-100:])
    # Try to parse the JSON to verify it's valid
    json.loads(content)
    print("JSON is valid!")

Verifying articles.json content:
File content length: 3040
First 100 characters: [
  {
    "content_id": "1.6272172",
    "content_headline": "CBC N.L. launches annual Make the Seas
Last 100 characters: r. The public can support the Community Food Sharing Association by making a donation online."
  }
]
JSON is valid!


In [14]:
# Cell 2: Install dependencies
print("Installing dependencies...")
!pip install fastapi uvicorn sentence-transformers faiss-cpu transformers torch numpy pydantic requests python-multipart

# Verify installations
import fastapi
import uvicorn
import sentence_transformers
import faiss
import transformers
import torch
import numpy
import pydantic
import requests

print("\nDependencies installed successfully!")
print(f"FastAPI version: {fastapi.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [4]:
# Cell 3: Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")



CUDA available: True
GPU: Tesla T4


In [5]:
# Cell 4: Set environment variable for Colab
import os
os.environ["COLAB"] = "1"







In [23]:
# Cell 5: Start FastAPI server
import os
import time
import requests
import json

print("Starting FastAPI server...")
print("Project directory:", project_dir)

# Print debug information
print("\nDebug Information:")
print(f"Python path: {os.environ.get('PYTHONPATH', 'Not set')}")
print(f"Current directory: {os.getcwd()}")
print(f"Project directory contents: {os.listdir(project_dir)}")
print(f"Data directory contents: {os.listdir(f'{project_dir}/data')}")

# Verify articles.json exists and is valid
articles_path = f'{project_dir}/data/articles.json'
print("\nVerifying articles.json...")
if os.path.exists(articles_path):
    try:
        with open(articles_path, 'r') as f:
            content = f.read()
            print(f"Articles file content length: {len(content)}")
            print(f"First 100 characters: {content[:100]}")
            print(f"Last 100 characters: {content[-100:]}")
            json.loads(content)  # Verify JSON is valid
            print("Articles.json is valid")
    except Exception as e:
        print(f"Error reading articles.json: {str(e)}")
else:
    print("Error: articles.json not found!")

# Kill any existing process on port 8000
print("\nKilling any existing process on port 8000...")
os.system('fuser -k 8000/tcp')
time.sleep(2)  # Wait for port to be freed

# Start the server with logging
print("\nStarting server...")
log_file = f'{project_dir}/server.log'

# Create a simple test script to verify the server
test_script = f'''import uvicorn
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {{"status": "ok", "message": "Test server is running"}}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

test_script_path = f'{project_dir}/test_server.py'
with open(test_script_path, 'w') as f:
    f.write(test_script)

# First try the test server
print("\nTesting with a simple server first...")
os.system(f'cd {project_dir} && python test_server.py > {log_file} 2>&1 &')
time.sleep(5)

# Check if test server is running
try:
    response = requests.get("http://localhost:8000/", timeout=5)
    print(f"Test server response: {response.status_code}")
    if response.status_code == 200:
        print("Test server is working!")
    else:
        print(f"Test server returned status {response.status_code}")
except Exception as e:
    print(f"Test server error: {str(e)}")

# Kill the test server
os.system('fuser -k 8000/tcp')
time.sleep(2)

# Now start the main server
print("\nStarting main server...")
server_cmd = f'cd {project_dir} && PYTHONPATH={project_dir} python src/api/main.py > {log_file} 2>&1'
os.system(f'{server_cmd} &')

# Wait for server to initialize
print("\nWaiting for server to initialize (30 seconds)...")
time.sleep(30)

# Check if server is running and show logs
try:
    response = requests.get("http://localhost:8000/", timeout=5)
    if response.status_code == 200:
        print("Server started successfully!")
        print("You can now run the test cell.")
    else:
        print(f"Warning: Server returned status code {response.status_code}")
        print("\nServer logs:")
        if os.path.exists(log_file):
            with open(log_file, 'r') as f:
                print(f.read())
        else:
            print("No log file found")
except Exception as e:
    print(f"Warning: Could not verify server status - {str(e)}")
    print("\nServer logs:")
    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            print(f.read())
    else:
        print("No log file found")

Starting FastAPI server...
Project directory: /content/drive/MyDrive/editorial_assistant

Debug Information:
Python path: /env/python
Current directory: /content
Project directory contents: ['test_server.py', 'editorial_assistant.html', 'run_server.py', 'src', 'data', 'server.log', 'editorial_assistant', 'requirements.txt']
Data directory contents: ['.DS_Store', 'articles.json', 'policies.txt']

Verifying articles.json...
Articles file content length: 1982
First 100 characters: [
  {
    "content_id": "article1",
    "content_headline": "CBC N.L. launches campaign to support f
Last 100 characters:  content and the importance of protecting sources who share information through social media."
  }
]
Articles.json is valid

Killing any existing process on port 8000...

Starting server...

Testing with a simple server first...
Test server response: 200
Test server is working!

Starting main server...

Waiting for server to initialize (30 seconds)...
Server started successfully!
You can now 

In [24]:
# Cell 6: Test the API
print("Starting API test...")

# Get the public URL from Colab
from google.colab.output import eval_js
public_url = eval_js("google.colab.kernel.proxyPort(8000)")
base_url = f"http://localhost:8000"  # Use localhost instead of public URL
api_url = f"{base_url}/api/qa"

print(f"Testing API at: {api_url}")

# Test questions
test_questions = [
    "What are the guidelines for using social media in news reporting?",
    "How should anonymous sources be handled?",
    "What are the requirements for writing headlines?",
    "Tell me about the Make the Season Kind campaign",
    "What are the guidelines for food bank donations?"
]

def test_qa_endpoint(question):
    print(f"\n{'='*80}")
    print(f"Testing question: {question}")
    print(f"{'='*80}")

    try:
        print("\nSending question to QA endpoint...")
        print(f"Request URL: {api_url}")

        response = requests.post(
            api_url,
            json={"question": question},
            timeout=30
        )

        print(f"Response status: {response.status_code}")

        if response.status_code == 200:
            result = response.json()
            print("\nAPI Response:")
            print("\nAnswer:")
            print("-" * 40)
            print(result["answer"])
            print("\nCitations:")
            print("-" * 40)
            for i, citation in enumerate(result["citations"], 1):
                print(f"\nCitation {i}:")
                print(f"Source: {citation['source']}")
                print(f"Text: {citation['text'][:200]}...")
        else:
            print(f"\nError: Server returned status code {response.status_code}")
            print("Response:", response.text)

    except requests.exceptions.RequestException as e:
        print(f"\nError making request: {str(e)}")
    except json.JSONDecodeError as e:
        print(f"\nError decoding response: {str(e)}")
        print("Response:", response.text)

# First check if server is running
print("\nChecking if server is running...")
try:
    response = requests.get(base_url, timeout=5)
    if response.status_code == 200:
        print("Server is running!")
        print("\nRunning test questions...")

        # Run all test questions
        for question in test_questions:
            test_qa_endpoint(question)
            time.sleep(2)  # Small delay between questions

    else:
        print(f"Server returned status {response.status_code}")
except Exception as e:
    print(f"Error connecting to server: {str(e)}")
    print("\nServer logs:")
    log_file = f'{project_dir}/server.log'
    if os.path.exists(log_file):
        with open(log_file, 'r') as f:
            print(f.read())

print("\nTest completed.")

Starting API test...
Testing API at: http://localhost:8000/api/qa

Checking if server is running...
Server is running!

Running test questions...

Testing question: What are the guidelines for using social media in news reporting?

Sending question to QA endpoint...
Request URL: http://localhost:8000/api/qa
Response status: 200

API Response:

Answer:
----------------------------------------
When creating content for social media:
1. Keep summaries concise and engaging
2. Use appropriate hashtags
3. Include key information in the first 280 characters
4. Maintain CBC's voice and tone
5. Ensure accuracy and fairness

Citations:
----------------------------------------

Citation 1:
Source: CBC Editorial Guidelines: Social Media
Text: When creating content for social media:
1. Keep summaries concise and engaging
2. Use appropriate hashtags
3. Include key information in the first 280 characters
4. Maintain CBC's voice and tone
5. En...

Citation 2:
Source: CBC Article: New guidelines for so

Starting API test...
Testing API at: https://8000-gpu-t4-s-1i1i6uj9ic6eb-a.us-west4-0.prod.colab.dev/api/qa
Sending question: What are the guidelines for using social media in news reporting?

Checking if server is running...
Root endpoint status: 404
Server is not responding correctly. Please check server logs.

Test completed.
